# 1, Load packages and initial setup

In [448]:
import pandas as pd
import numpy as np
import csv
import scipy as sp

from bs4 import BeautifulSoup
import json

import urllib
import requests
from sklearn import preprocessing

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [423]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# 2, Grab player index and player table from the link through web-scrapping

In [450]:
## Check if the link is working
url = 'https://stats.nba.com/stats/playercareerstats?PerMode=PerGame&PlayerID=977'
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:65.0) Gecko/20100101 Firefox/65.0',
              'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
               'Accept-Language': 'en-US,en;q=0.5',
           'Accept-Encoding': 'gzip, deflate, br',
               'Cache-Control': 'max-age=0',
          'Connection': 'keep-alive',
           'Upgrade-Insecure-Requests': '1',
               'Referer': 'http://www.baidu.com/'}
response2 = requests.get(url, headers=headers)
response2.status_code
#response.headers.keys()

200

In [113]:
## Check if the link is working
url = 'https://stats.nba.com/stats/commonallplayers?LeagueId=00&Season=2016-17&IsOnlyCurrentSeason=0'
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:65.0) Gecko/20100101 Firefox/65.0',
              'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
               'Accept-Language': 'en-US,en;q=0.5',
           'Accept-Encoding': 'gzip, deflate, br',
               'Cache-Control': 'no-cache, no-store, must-revalidate',
          'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
           'Host': 'stats.nba.com',
               'Referer': 'http://www.baidu.com/'}

response = requests.get(url, headers=headers)
response.status_code


200

In [278]:
## Extract player Index Table
soup = BeautifulSoup(response.text, 'lxml') # Parse the HTML as a string
obj = json.loads(soup.get_text())  

In [326]:
player_idx = pd.DataFrame(obj['resultSets'][0]['rowSet'])
player_idx.columns = obj['resultSets'][0]['headers']
player_idx

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,GAMES_PLAYED_FLAG
0,76001,"Abdelnaby, Alaa",Alaa Abdelnaby,0,1990,1994,HISTADD_alaa_abdelnaby,0,,,,,Y
1,76002,"Abdul-Aziz, Zaid",Zaid Abdul-Aziz,0,1968,1977,HISTADD_zaid_abdul-aziz,0,,,,,Y
2,76003,"Abdul-Jabbar, Kareem",Kareem Abdul-Jabbar,0,1969,1988,HISTADD_kareem_abdul-jabbar,0,,,,,Y
3,51,"Abdul-Rauf, Mahmoud",Mahmoud Abdul-Rauf,0,1990,2000,mahmoud_abdul-rauf,0,,,,,Y
4,1505,"Abdul-Wahad, Tariq",Tariq Abdul-Wahad,0,1997,2003,tariq_abdul-wahad,0,,,,,Y
5,949,"Abdur-Rahim, Shareef",Shareef Abdur-Rahim,0,1996,2007,shareef_abdur-rahim,0,,,,,Y
6,76005,"Abernethy, Tom",Tom Abernethy,0,1976,1980,HISTADD_tom_abernethy,0,,,,,Y
7,76006,"Able, Forest",Forest Able,0,1956,1956,HISTADD_frosty_able,0,,,,,Y
8,76007,"Abramovic, John",John Abramovic,0,1946,1947,HISTADD_brooms_abramovic,0,,,,,Y
9,203518,"Abrines, Alex",Alex Abrines,1,2016,2018,alex_abrines,1610612760,Oklahoma City,Thunder,OKC,thunder,Y


In [367]:
## Because the business goal is to check how unbalanced a team lineup is, I will focus on active players 
active_player = player_idx[player_idx.ROSTERSTATUS == 1]
active_player_id = list(active_player.PERSON_ID)
active_player.head()

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,GAMES_PLAYED_FLAG
9,203518,"Abrines, Alex",Alex Abrines,1,2016,2018,alex_abrines,1610612760,Oklahoma City,Thunder,OKC,thunder,Y
14,203112,"Acy, Quincy",Quincy Acy,1,2012,2018,quincy_acy,1610612751,Brooklyn,Nets,BKN,nets,Y
21,203500,"Adams, Steven",Steven Adams,1,2013,2018,steven_adams,1610612760,Oklahoma City,Thunder,OKC,thunder,Y
27,201167,"Afflalo, Arron",Arron Afflalo,1,2007,2017,arron_afflalo,1610612758,Sacramento,Kings,SAC,kings,Y
32,201582,"Ajinca, Alexis",Alexis Ajinca,1,2008,2017,alexis_ajinca,1610612740,New Orleans,Pelicans,NOP,pelicans,Y


In [402]:
## Extract players' information
## Note: Have to setup timeout parameter appropriately, such that we are not blocked by NBA.com. 
## Thus, it takes a little more time to grab data from there.
player_stats = pd.DataFrame()
#i = 0
for x in active_player_id:
#    i += 1
    url = 'https://stats.nba.com/stats/playercareerstats?PerMode=PerGame&PlayerID='+ str(x)
    headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:65.0) Gecko/20100101 Firefox/65.0',
                  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
                   'Accept-Language': 'en-US,en;q=0.5',
               'Accept-Encoding': 'gzip, deflate, br',
                   'Cache-Control': 'max-age=0',
              'Connection': 'keep-alive',
               'Upgrade-Insecure-Requests': '1',
                   'Referer': 'http://www.baidu.com/'}
    response2 = requests.get(url, headers=headers, timeout = 1)
    soup = BeautifulSoup(response2.text, 'lxml') # Parse the HTML as a string
    obj = json.loads(soup.get_text())
    temp = pd.DataFrame(obj['resultSets'][0]['rowSet'])
    temp.columns = obj['resultSets'][0]['headers']
    player_stats = pd.concat([player_stats, temp])
#    if i %100 == 0:
#        print(i)

In [415]:
player_stats = player_stats.reset_index(drop = True)

In [930]:
df.head(10)

,PLAYER_ID,TEAM_ABBREVIATION,FG_PCT,FG3_PCT,FT_PCT,fg_pts,FGM_pm,FGA_pm,FG3M_pm,FG3A_pm,FTM_pm,FTA_pm,OREB_pm,DREB_pm,REB_pm,AST_pm,STL_pm,BLK_pm,TOV_pm,PF_pm,PTS_pm
0,203518,OKC,0.393,0.381,0.898,0.900000,0.129032,0.322581,0.090323,0.232258,0.038710,0.045161,0.019355,0.064516,0.083871,0.038710,0.032258,0.006452,0.032258,0.109677,0.387097
9,203112,DAL,0.294,0.143,0.667,0.863636,0.100000,0.350000,0.025000,0.150000,0.037500,0.062500,0.037500,0.125000,0.162500,0.000000,0.000000,0.000000,0.037500,0.187500,0.275000
10,203112,BKN,0.425,0.434,0.754,0.800000,0.125786,0.301887,0.069182,0.163522,0.081761,0.113208,0.037736,0.176101,0.207547,0.037736,0.025157,0.031447,0.037736,0.113208,0.408805
11,203112,TOT,0.412,0.411,0.750,0.793103,0.122449,0.306122,0.068027,0.163265,0.081633,0.108844,0.034014,0.170068,0.204082,0.034014,0.027211,0.027211,0.040816,0.122449,0.394558
17,203500,OKC,0.571,0.000,0.611,0.823009,0.157191,0.274247,0.000000,0.000000,0.066890,0.107023,0.117057,0.140468,0.257525,0.036789,0.036789,0.033445,0.060201,0.080268,0.377926
31,201167,SAC,0.440,0.411,0.892,0.833333,0.115830,0.266409,0.038610,0.096525,0.054054,0.057915,0.003861,0.073359,0.077220,0.050193,0.011583,0.003861,0.027027,0.065637,0.324324
41,201582,NOP,0.500,0.000,0.725,0.867925,0.153333,0.306667,0.000000,0.006667,0.046667,0.066667,0.080000,0.226667,0.300000,0.020000,0.033333,0.040000,0.053333,0.133333,0.353333
50,202332,MIN,0.523,0.000,0.682,0.882353,0.081395,0.162791,0.000000,0.000000,0.023256,0.046512,0.093023,0.197674,0.290698,0.046512,0.046512,0.046512,0.034884,0.162791,0.197674
62,200746,SAS,0.477,0.411,0.812,0.820809,0.212963,0.450617,0.009259,0.024691,0.095679,0.117284,0.074074,0.151235,0.225309,0.058642,0.018519,0.037037,0.043210,0.067901,0.533951
72,202730,IND,0.458,0.000,0.697,0.862069,0.090909,0.195804,0.000000,0.000000,0.027972,0.034965,0.118881,0.132867,0.251748,0.062937,0.020979,0.027972,0.034965,0.090909,0.202797


# 3, Data pre-process

In [445]:
## In order to do K-means, we need to pick the right data. But raw data is a time-series data at player level.
## We could process it in 2 ways
## Method 1: Select players' stats in certain season, such as 2016-2017
## Method 2: Generate average stats across seasons, such as average between 2014 and 2017
## For this exercise, I just use method 1 by simplicity
df = player_stats[player_stats.SEASON_ID == '2016-17'].copy()
df.describe(include = 'all')

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
count,5.470000e+02,547,547,5.470000e+02,547,547.000000,547.000000,547.000000,547.000000,547.000000,547.000000,547.000000,547.000000,547.000000,547.000000,547.000000,547.000000,547.000000,547.000000,547.000000,547.000000,547.000000,547.000000,547.000000,547.000000,547.000000,547.000000
unique,NaN,1,1,NaN,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,2016-17,00,NaN,TOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,547,547,NaN,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,5.446641e+05,NaN,NaN,1.466335e+09,NaN,26.751371,51.308958,23.882998,20.270750,3.195978,7.039854,0.449594,0.797989,2.250457,0.286181,1.431261,1.877697,0.724927,0.854113,2.769104,3.620658,1.824132,0.632358,0.410969,1.127239,1.726508,8.615539
std,6.380494e+05,NaN,NaN,4.603773e+08,NaN,4.283067,25.075154,27.897921,8.626262,2.073154,4.382936,0.089656,0.752198,1.926425,0.150771,1.420865,1.723702,0.176564,0.772298,1.791989,2.425859,1.733257,0.410747,0.485682,0.771213,0.734231,5.851120
min,1.713000e+03,NaN,NaN,0.000000e+00,NaN,19.000000,1.000000,0.000000,1.800000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.015865e+05,NaN,NaN,1.610613e+09,NaN,24.000000,29.500000,1.000000,14.100000,1.700000,3.800000,0.403000,0.200000,0.700000,0.244500,0.600000,0.800000,0.667000,0.300000,1.500000,2.000000,0.700000,0.300000,0.100000,0.600000,1.200000,4.600000
50%,2.030940e+05,NaN,NaN,1.610613e+09,NaN,26.000000,58.000000,11.000000,20.000000,2.700000,6.000000,0.444000,0.600000,1.800000,0.333000,1.000000,1.400000,0.764000,0.600000,2.400000,3.000000,1.200000,0.600000,0.300000,0.900000,1.700000,7.100000
75%,1.626146e+06,NaN,NaN,1.610613e+09,NaN,29.000000,74.000000,43.000000,27.000000,4.200000,9.200000,0.490000,1.300000,3.500000,0.375000,1.800000,2.400000,0.832000,1.200000,3.500000,4.700000,2.300000,0.900000,0.500000,1.500000,2.200000,11.000000


In [446]:
## Feature engineering
## Because the business goal is to create a signal on how unbalanced a team lineup is (aggressive or defensive),
## I propose to refer to offensive/defensive rating formula
## 1, Defensive Player Rating = (Players Points*Total FG%) + Opponents Differential= 1/5 of possessions - 
## Times Fouled+ FTM* FT% * OAPOW( Official Adjusted Players Offensive Withstand
## 2, Offensive Production Rating = (Points Produced / Individual Possessions) x OAPOW × PPG + FTM/FT * 3pt% + FG%

# Conclusion:
## 1, In order to capture the features used in defensive/offensive rating, we need almost all features in the tables 
## 2, But still we could drop some ones to reduce dimensionality, such as GP, GS, MIN, FGM, FGA, FG3M, FG3A, FTM, FTA
## 3, We don't have team-level data, which is the most relevant to team unbalance. Instead, we use player-level data as approximates
## 4, new features (due to time constraints, I didn't do much feature engineering):
## 4.1 percent of pts made by FG
## 4.2 Normalize the stats by minutes played -- stats per min  

df['fg_pts'] = 1-(df['FTM']/df['PTS'])
df['fg_pts'] = df['fg_pts'].fillna(0)

for x in [ 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 
          'PTS']:
    df[x+'_pm'] = df[x]/df['MIN']

df = df.drop(['SEASON_ID', 'LEAGUE_ID', 'TEAM_ID', 'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG3M',
                        'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
            axis = 1)
df.columns

Index(['PLAYER_ID', 'TEAM_ABBREVIATION', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'fg_pts', 'FGM_pm', 'FGA_pm', 'FG3M_pm', 'FG3A_pm', 'FTM_pm', 'FTA_pm', 'OREB_pm', 'DREB_pm', 'REB_pm', 'AST_pm', 'STL_pm', 'BLK_pm', 'TOV_pm', 'PF_pm', 'PTS_pm'], dtype='object')

In [881]:
## Normalization
## There are different ways to normalize/standardize the data. Here we use z-score

df0 = df[['FG_PCT', 'FG3_PCT', 'FT_PCT', 'fg_pts', 'FGM_pm', 'FGA_pm', 'FG3M_pm', 
                                      'FG3A_pm', 'FTM_pm', 'FTA_pm', 'OREB_pm', 'DREB_pm', 'REB_pm', 'AST_pm',
                                      'STL_pm', 'BLK_pm', 'TOV_pm', 'PF_pm', 'PTS_pm']]

scalar = preprocessing.StandardScaler().fit(df0)


df_std = scalar.transform(df0)

# 4, K-Means

In [882]:
for k in range (1, 11):
 
    # Create a kmeans model on our data, using k clusters.  random_state helps ensure that the algorithm returns the same results each time.
    kmeans_model = KMeans(n_clusters=k, random_state=1).fit(df_std)

    # These are our fitted labels for clusters -- the first cluster has label 0, and the second has label 1.
    #labels = kmeans_model.labels_
 
    # Sum of distances of samples to their closest cluster center
    interia = kmeans_model.inertia_
    print ("k:",k, " cost:", interia)

k: 1  cost: 10393.0
k: 2  cost: 8259.541272907449
k: 3  cost: 6997.4774941829
k: 4  cost: 6509.344516956635
k: 5  cost: 6121.251072534678
k: 6  cost: 5802.080905108382
k: 7  cost: 5511.773922764162
k: 8  cost: 5308.1791687328805
k: 9  cost: 5003.599153562102
k: 10  cost: 4844.8183112011875


In [883]:
## According to the check above, optimal K = 3
km = KMeans(3, random_state=1)

clusters = km.fit_predict(df_std)

clusters

array([1, 1, 1, 1, 2, 1, 2, 2, 0, 2, 2, 1, 1, 0, 1, 1, 1, 1, 0, 0, 2, 1,
       1, 2, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 2, 0, 0, 2, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 2, 1, 2, 0, 0, 0, 0, 2, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 0,
       1, 2, 1, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 2, 2, 0, 2, 1, 1, 1, 1, 2, 1, 2, 1, 0, 2, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 2, 2, 0, 2, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2,
       0, 0, 0, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 2, 0, 0, 1,
       2, 1, 1, 1, 2, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1,
       1, 2, 0, 2, 1, 1, 0, 1, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 2, 2, 0, 1, 1, 1, 1, 0, 0,
       2, 1, 1, 2, 2, 1, 1, 2, 1, 2, 0, 0, 1, 1, 1, 2, 1, 1, 2, 0, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 0, 1, 1, 1, 2, 1, 0, 0,

In [927]:
output0 = pd.concat([df[['PLAYER_ID', 'TEAM_ABBREVIATION']].reset_index(drop = True),
                    pd.DataFrame(clusters)], axis = 1)
output0.columns = ['PLAYER_ID', 'TEAM_ABBREVIATION', 'cluster']
output = pd.merge(output0,
                 player_idx[['PERSON_ID', 'DISPLAY_FIRST_LAST']],
                 how = 'left',
                 left_on = 'PLAYER_ID',
                 right_on = 'PERSON_ID')

output['first_name'] = [x.split(" ")[0] for  x in output['DISPLAY_FIRST_LAST']]
output['last_name'] = ["-".join(x.split(" ")[1:]) for  x in output['DISPLAY_FIRST_LAST']]

output = output[['PLAYER_ID','first_name', 'last_name','TEAM_ABBREVIATION', 'cluster']]
output[output.cluster == 0]

,PLAYER_ID,first_name,last_name,TEAM_ABBREVIATION,cluster
8,200746,LaMarcus,Aldridge,SAS,0
13,1626147,Justin,Anderson,DAL,0
18,203507,Giannis,Antetokounmpo,MIL,0
19,2546,Carmelo,Anthony,NYK,0
29,200826,J.J.,Barea,DAL,0
30,203084,Harrison,Barnes,DAL,0
34,203115,Will,Barton,DEN,0
36,201587,Nicolas,Batum,CHA,0
37,201573,Jerryd,Bayless,PHI,0
40,203078,Bradley,Beal,WAS,0


# 5, Insights

In [ ]:
## Notes: How to explain the results
### Who are in each cluster
### cluster 0: LeBron James, Stephen Curry, James Harden, Kevin Durant, Giannis Antetokounmpo,...
### cluster 1: Vince Carter, Paul Pierce, Manu Ginobilli,...
### cluster 2: Steven Adams, Clint Capela, Paul Gasol,...

### cluster 0: offenseive players
### cluster 1: swingman
### cluster 2: defensive players

### If we average players cluster numbers by team, we will get team-level balance. 
### The higher the number, the more defensive the team is

In [887]:
## Obtain the team-level score
balance = pd.merge(output0[['PLAYER_ID','TEAM_ABBREVIATION', 'cluster']], 
                  player_stats.loc[player_stats.SEASON_ID == '2016-17', ['PLAYER_ID', 'TEAM_ABBREVIATION', 
                                                                         'MIN', 'GP']],
                  how = 'left',
                  on = ['PLAYER_ID', 'TEAM_ABBREVIATION'])

balance['cluster'] = balance['cluster']*(balance['GP']/82) ## weighted by # games played
balance = balance[['TEAM_ABBREVIATION', 
                  'cluster']].groupby('TEAM_ABBREVIATION').mean().reset_index().sort_values('cluster',
                                                                                            ascending = False)
balance.reset_index(drop = True)

,TEAM_ABBREVIATION,cluster
0,SAS,0.851220
1,GSW,0.810213
2,LAL,0.789096
3,MEM,0.786585
4,BOS,0.786179
5,IND,0.779675
6,UTA,0.775610
7,DET,0.737398
8,TOR,0.730945
9,ORL,0.729421


In [ ]:
## Model predicted ranking are close to published Net Rating = Offensive Rating - Defensive Rating
# 1	Golden State Warriors
# 2	San Antonio Spurs
# 3	Houston Rockets
# 4	LA Clippers
# 5	Utah Jazz
# 6	Toronto Raptors
# 7	Cleveland Cavaliers
# 8	Boston Celtics
# 9	Washington Wizards
# 10	Miami Heat
# 11	Oklahoma City Thunder
# 12	Memphis Grizzlies
# 13	Denver Nuggets
# 14	Chicago Bulls
# 15	Charlotte Hornets
# 16	Indiana Pacers
# 17	Milwaukee Bucks
# 18	Portland Trail Blazers
# 19	Atlanta Hawks
# 20	Minnesota Timberwolves
# 21	Detroit Pistons
# 22	New Orleans Pelicans
# 23	Dallas Mavericks
# 24	New York Knicks
# 25	Sacramento Kings
# 26	Phoenix Suns
# 27	Philadelphia 76ers
# 28	Brooklyn Nets
# 29	Orlando Magic
# 30	Los Angeles Lakers

## Note: 
## Lakers are the exception. Actually, Lakers defense is significanly worse than rest of the league in 2016-17 season
## But in K-Means, we give all features same weights. 
## In order to get aligned with net rating, we'd give defensive features more weights.

# 6, Final results and export

In [923]:
## Final output and export
output_final = output[['cluster', 'PLAYER_ID','first_name', 'last_name']].sort_values(['cluster', 'PLAYER_ID'])
output_final.index = output_final['cluster']
output_final =  output_final[['PLAYER_ID','first_name', 'last_name']]
output_final = dict(output_final.groupby(level=0).apply(lambda x: x.to_json(orient='records')))
print (output_final)

{0: '[{"PLAYER_ID":1717,"first_name":"Dirk","last_name":"Nowitzki"},{"PLAYER_ID":2544,"first_name":"LeBron","last_name":"James"},{"PLAYER_ID":2546,"first_name":"Carmelo","last_name":"Anthony"},{"PLAYER_ID":2548,"first_name":"Dwyane","last_name":"Wade"},{"PLAYER_ID":2563,"first_name":"Dahntay","last_name":"Jones"},{"PLAYER_ID":101108,"first_name":"Chris","last_name":"Paul"},{"PLAYER_ID":101141,"first_name":"Ersan","last_name":"Ilyasova"},{"PLAYER_ID":101141,"first_name":"Ersan","last_name":"Ilyasova"},{"PLAYER_ID":101150,"first_name":"Lou","last_name":"Williams"},{"PLAYER_ID":101150,"first_name":"Lou","last_name":"Williams"},{"PLAYER_ID":101150,"first_name":"Lou","last_name":"Williams"},{"PLAYER_ID":200746,"first_name":"LaMarcus","last_name":"Aldridge"},{"PLAYER_ID":200752,"first_name":"Rudy","last_name":"Gay"},{"PLAYER_ID":200755,"first_name":"JJ","last_name":"Redick"},{"PLAYER_ID":200768,"first_name":"Kyle","last_name":"Lowry"},{"PLAYER_ID":200794,"first_name":"Paul","last_name":"Mill